In [1]:
import os
import numpy as np
import keras
from keras.applications import ResNet50V2
from keras import models
from keras import layers
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras import optimizers
import matplotlib.pyplot as plt
import cv2
from keras.layers import Dropout
from tensorflow.keras import backend as K
import tensorflow as tf
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
import csv
import pandas as pd

Using TensorFlow backend.
C:\Users\idong\Anaconda3\envs\tens_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\idong\Anaconda3\envs\tens_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\idong\Anaconda3\envs\tens_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\idong\Anaconda3\envs\tens_gpu

In [2]:
base_dir = r'D:\dataset\dental\data'

data = np.load(base_dir + '\\data_KNU_Diorco_so_imgs.npy')
label_csv= open(base_dir + '\\data_KNU_Diorco_so_img_labels.csv', 'r')
label = csv.reader(label_csv)

# 모델 1,2,3,4 데이터셋

In [3]:
right_label = np.load(base_dir + '\\right_label.npy')
left_label = np.load(base_dir + '\\left_label.npy')

In [4]:
right = data[:,:,:, 0:3]
left = data[:,:,:, 3:7]

patient_data = []
patient_data.extend(left)
patient_data.extend(right)
patient_data = np.array(patient_data)

patient_labels = []
patient_labels.extend(left_label)
patient_labels.extend(right_label)
patient_labels = np.array(patient_labels)

In [5]:
train_data, test_data, train_label, test_label= train_test_split(patient_data, patient_labels, test_size = 0.2, random_state = 33, shuffle = True)
train_data, val_data, train_label, val_label = train_test_split(train_data, train_label, test_size = 0.2, random_state = 33, shuffle = True)

In [6]:
#원 핫 인코딩을 위해 label을 바꿈
one_hot_train_labels = to_categorical(train_label - 1)
one_hot_validation_labels = to_categorical(val_label  - 1)
one_hot_test_labels = to_categorical(test_label - 1)

In [7]:
train_data = train_data.astype(np.float32) / 127.5 - 1
test_data = test_data.astype(np.float32) / 127.5 - 1
val_data = val_data.astype(np.float32) / 127.5 - 1

# 모델5 data_set

In [3]:
right_label = np.load(base_dir + '\\right_label.npy')
left_label = np.load(base_dir + '\\left_label.npy')

right = data[:,:,:, 0:3]
left = data[:,:,:, 3:7]

patient_datas = np.load(r'D:\dataset\dental\data\preprocessing\patient_data_2.npy', allow_pickle=True)
patient_labels = np.load(r'D:\dataset\dental\data\preprocessing\patient_label_2.npy', allow_pickle=True)

train_data, test_data, train_label, test_label= train_test_split(patient_datas, patient_labels, test_size = 0.2, random_state = 123)
train_data, val_data, train_label, val_label = train_test_split(train_data, train_label, test_size = 0.2, random_state = 123)

train_label = np.array(train_label)
test_label = np.array(test_label)
val_label = np.array(val_label)

#원 핫 인코딩을 위해 label을 바꿈
one_hot_train_labels = to_categorical(train_label - 1)
one_hot_validation_labels = to_categorical(val_label - 1)
one_hot_test_labels = to_categorical(test_label - 1)

real_train = []
real_val = []
real_test = []

def flat_datas(data, data_name):
    for m in range(len(data)):
        for j in range(4):
            data_name.append(data[m][j])

flat_datas(train_data, real_train)
flat_datas(val_data, real_val)
flat_datas(test_data, real_test)

train_data = np.array(real_train)
val_data = np.array(real_val)
test_data = np.array(real_test)

real_train_l = []
real_val_l = []
real_test_l = []

def flat_labels(label, label_name):
    for m in range(len(label)):
        for j in range(4):
            label_name.append(label[m][j])

flat_labels(one_hot_train_labels, real_train_l)
flat_labels(one_hot_validation_labels, real_val_l)
flat_labels(one_hot_test_labels, real_test_l)

one_hot_train_labels = np.array(real_train_l)
one_hot_val_labels = np.array(real_val_l)
one_hot_test_labels = np.array(real_test_l)

train_data = train_data.astype(np.float32) / 127.5 - 1
test_data = test_data.astype(np.float32) / 127.5 - 1
val_data = val_data.astype(np.float32) / 127.5 - 1

## 1번 모델 test

In [6]:
model = models.load_model('real_dental_ResNet50V2_2.h5')
model.load_weights('real_dental_ResNet50V2_2.h5')

In [9]:
model = models.load_model('real_dental_ResNet50V2_3.h5')
model.load_weights('real_dental_ResNet50V2_3.h5')

In [10]:
test_loss, test_acc = model.evaluate(test_data, one_hot_test_labels)
print('test acc:' , test_acc)

324/324 [==============================] - 22s 69ms/step
test acc: 0.5524691343307495


## 2번 모델 test

In [17]:
model = models.load_model('real2_dental_ResNet50V2_2.h5')
model.load_weights('real2_dental_ResNet50V2_2.h5')

In [18]:
test_loss, test_acc = model.evaluate(test_data, one_hot_test_labels)
print('test acc:' , test_acc)

324/324 [==============================] - 17s 51ms/step
test acc: 0.5987654328346252


In [8]:
model = models.load_model('real2_dental_ResNet50V2_3.h5')
model.load_weights('real2_dental_ResNet50V2_3.h5')

In [9]:
test_loss, test_acc = model.evaluate(test_data, one_hot_test_labels)
print('test acc:' , test_acc)

324/324 [==============================] - 22s 68ms/step
test acc: 0.5925925970077515


## 3번 모델 test

In [19]:
model = models.load_model('real3_dental_ResNet50V2_3.h5')
model.load_weights('real3_dental_ResNet50V2_3.h5')

In [20]:
test_loss, test_acc = model.evaluate(test_data, one_hot_test_labels)
print('test acc:' , test_acc)

324/324 [==============================] - 16s 50ms/step
test acc: 0.6358024477958679


# 4번 모델 test

In [9]:
model = models.load_model('real4_dental_ResNet50V2_2.h5')
model.load_weights('real4_dental_ResNet50V2_2.h5')

In [10]:
test_loss, test_acc = model.evaluate(test_data, one_hot_test_labels)
print('test acc:' , test_acc)

324/324 [==============================] - 23s 71ms/step
test acc: 0.6388888955116272


# 5번 모델 test

In [4]:
model = models.load_model('real5_dental_ResNet50V2_3.h5')
model.load_weights('real5_dental_ResNet50V2_3.h5')

In [5]:
test_loss, test_acc = model.evaluate(test_data, one_hot_test_labels)
print('test acc:' , test_acc)

324/324 [==============================] - 23s 71ms/step
test acc: 0.5586419701576233
